In [1]:
# imports necessary packages

import os
import cv2
import PIL
from glob import glob
import pandas as pd
import numpy as np

In [2]:
# Defines a function to crop the turtles by blacking out the background

def turtle_crop(image_file, mask_file):
    # Reads the image and mask using OpenCV
    image = cv2.imread(image_file)
    mask = cv2.imread(mask_file)
    
    # Ensure image and mask have the same dimensions
    assert image.shape[:2] == mask.shape[:2], "Image and mask must have the same dimensions"
    
    # Convert mask to binary (0 and 1 in this case)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    
    # Find contours of the object in the mask, returns array of points and contour hierarchy
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Find the largest contour (object of interest) by seeing which one contains the highest area
    contour = max(contours, key=cv2.contourArea)
    
    # Create a blank mask with the same dimensions as the input image
    crop_mask = np.zeros_like(image[:, :, :])
    
    # Draw all contours on the mask, contour border has an rgb = 1,1,1 and the inside of the contour is flled, not just drawing a line.
    cv2.drawContours(crop_mask, [contour], -1, (1,1,1), cv2.FILLED)
    
    # Tensor multiplication of the mask and the input image to crop the object of interest
    cropped_image = image * crop_mask[:, :, :]
    
    return cropped_image

In [3]:
def turtle_dir_crop(image_list, masks_list, save_dir):
    
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    
    counter = 0
    for img, mask in zip(image_list, masks_list):
        counter += 1
        img_name = img.split("/")[-1]
        cropped_img = turtle_crop(img, mask)
        save_path = os.path.join(save_dir, img_name)
        cv2.imwrite(save_path, cropped_img)
        if counter % 10 == 0:
            percent = round((counter/len(image_list))*100, 2)
            print("Processed {percent}% of the images".format(percent = percent))
    
    return print("All images cropped successfully")

In [4]:
# Specifies path of notebook

path = os.path.abspath(os.path.dirname("__file__"))

In [5]:
# Specifies paths of images and masks relative to the notebook (or path specified above)

images_path = os.path.join(path, "turtle_imgs")
masks_path = os.path.join(path, "pred_masks")
save_dir  = os.path.join(path, "cropped_imgs")

In [6]:
# Defines a list of the images and masks in their respective directories

images_list = glob(images_path + "/*.png")
masks_list = glob(masks_path + "/*.png")

In [7]:
# Modifies image and mask list to ensure only images that have a mask are used

images_list = [img.split("/")[-1] for img in images_list]
masks_list = [img.split("/")[-1] for img in masks_list]

# Modifies image list so that only images with a mask are processed

images_list = [img for img in images_list if img in masks_list]

In [8]:
# Appends the corresponding paths to the image and mask paths

images_list = [images_path + "/" + img for img in images_list]
masks_list = [masks_path + "/" + img for img in masks_list]

In [9]:
# Combines the predicted masks with the corresponding images

turtle_dir_crop(image_list=images_list, masks_list=masks_list, save_dir=save_dir)

Processed 50.0% of the images
Processed 100.0% of the images
All images cropped successfully
